## Loading Data 2010-2021

In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# loading data
df = pd.read_csv("../CSV/2010-2021.csv")
# drop unused columns
df.drop('LOWEST SALE PRICE', axis=1, inplace=True)
df.drop('HIGHEST SALE PRICE', axis=1, inplace=True)
# create grouping of data using specified column then
# creates a copy of specified entry within the column group
borough = groupby_borough = df.groupby(['BOROUGH']).get_group(('STATEN ISLAND',)).copy()

# group further by home type
groupby_home = borough.groupby(['TYPE OF HOME'])
one = groupby_home.get_group(('01 ONE FAMILY HOMES',)).copy()
two = groupby_home.get_group(('02 TWO FAMILY HOMES',)).copy()
three = groupby_home.get_group(('03 THREE FAMILY HOMES',)).copy()

## Setting up Model

In [3]:
# group home type into neighborhood
hood_names_one = one['NEIGHBORHOOD'].unique()
hood_names_two = two['NEIGHBORHOOD'].unique()
hood_names_three = three['NEIGHBORHOOD'].unique()

# get unique neighborhood names
groupby_hood_one = one.groupby(['NEIGHBORHOOD'])
groupby_hood_two = two.groupby(['NEIGHBORHOOD'])
groupby_hood_three = three.groupby(['NEIGHBORHOOD'])

# run the code using print(h) to get problem data and we hard code this
# this way the CVS file doesn't get modified continuously
# not enough data, to split the data 80/20
small_sample_one = ["NEW BRIGHTON-ST. GEORGE"] 
small_sample_two = ["TODT HILL", "BLOOMFIELD", "FRESH KILLS"] 
small_sample_three = ["ARDEN HEIGHTS", "HUGUENOT", "GRYMES HILL", "CLOVE LAKES", 
                "OAKWOOD-BEACH", "ELTINGVILLE", "NEW DORP-HEIGHTS", "PRINCES BAY", 
                "SILVER LAKE", "TRAVIS", "GRASMERE", "PLEASANT PLAINS", "WOODROW"]

# prediction model - ONE
dic_one = {}
for h in hood_names_one:
    temp = groupby_hood_one.get_group((h,)).copy()
    # print(h)
    if h in small_sample_one:
        dic_one[h] = temp['AVERAGE SALE PRICE'].item()
    else:
        X = temp[['YEAR', 'MEDIAN SALE PRICE', 'NUMBER OF SALES']] # independent variables table
        y = temp['AVERAGE SALE PRICE']
    
        # split the data 80/20
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
        # fit model
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # compare to test data
        y_pred = model.predict(X_test)
        dic_one[h] = y_pred[0]


# prediction model - TWO
dic_two = {}
for h in hood_names_two:
    temp = groupby_hood_two.get_group((h,)).copy()
    # print(h)
    if h in small_sample_two:
        dic_two[h] = temp['AVERAGE SALE PRICE'].item()
    else:
        X = temp[['YEAR', 'MEDIAN SALE PRICE', 'NUMBER OF SALES']]
        y = temp['AVERAGE SALE PRICE']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        dic_two[h] = y_pred[0]


# prediction model - THREE
dic_three = {}
for h in hood_names_three:
    temp = groupby_hood_three.get_group((h,)).copy()
    # print(h)
    if h in small_sample_three:
        dic_three[h] = temp['AVERAGE SALE PRICE'].item()
    else:
        X = temp[['YEAR', 'MEDIAN SALE PRICE', 'NUMBER OF SALES']]
        y = temp['AVERAGE SALE PRICE']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        dic_three[h] = y_pred[0]


## Comparing Model to Data 2022

In [7]:
# loading real data
df22 = pd.read_csv("../CSV/2022.csv")
df22.drop('NUMBER OF SALES', axis=1, inplace=True)
df22.drop('LOWEST SALE PRICE', axis=1, inplace=True)
df22.drop('MEDIAN SALE PRICE', axis=1, inplace=True)
df22.drop('HIGHEST SALE PRICE', axis=1, inplace=True)
df22.drop('YEAR', axis=1, inplace=True)
test = groupby_borough = df22.groupby(['BOROUGH']).get_group(('STATEN ISLAND',)).copy()
test.drop('BOROUGH', axis=1, inplace=True)

test['PREDICT'] = "-"
test['DELTA'] = "0"

# use 3 conditionals because not all targeted neighborhoods from the 2022 may appear in 2010-2021
for i, row in test.iterrows():
    if row['TYPE OF HOME'] == '01 ONE FAMILY HOMES' and row['NEIGHBORHOOD'] in dic_one:
        test.at[i, 'PREDICT'] = int(dic_one[row['NEIGHBORHOOD']])
        test.at[i, 'DELTA'] = int(row['AVERAGE SALE PRICE']) - int(dic_one[row['NEIGHBORHOOD']])
    elif row['TYPE OF HOME'] == '02 TWO FAMILY HOMES' and row['NEIGHBORHOOD'] in dic_two:
        test.at[i, 'PREDICT'] = int(dic_two[row['NEIGHBORHOOD']])
        test.at[i, 'DELTA'] = int(row['AVERAGE SALE PRICE']) - int(dic_two[row['NEIGHBORHOOD']])
    elif row['TYPE OF HOME'] == '03 THREE FAMILY HOMES' and row['NEIGHBORHOOD'] in dic_three:
        test.at[i, 'PREDICT'] = int(dic_three[row['NEIGHBORHOOD']])
        test.at[i, 'DELTA'] = int(row['AVERAGE SALE PRICE']) - int(dic_three[row['NEIGHBORHOOD']])

# displays all 128 rows
pd.options.display.max_rows = 128
test

,NEIGHBORHOOD,TYPE OF HOME,AVERAGE SALE PRICE,PREDICT,DELTA
477,ANNADALE,01 ONE FAMILY HOMES,954709,649104,305605
478,ANNADALE,02 TWO FAMILY HOMES,972797,771265,201532
479,ANNADALE,03 THREE FAMILY HOMES,995900,825000,170900
480,ARDEN HEIGHTS,01 ONE FAMILY HOMES,518603,372550,146053
481,ARDEN HEIGHTS,02 TWO FAMILY HOMES,904782,549002,355780
482,ARROCHAR,01 ONE FAMILY HOMES,684467,536936,147531
483,ARROCHAR,02 TWO FAMILY HOMES,744893,574833,170060
484,ARROCHAR-SHORE ACRES,01 ONE FAMILY HOMES,621032,502342,118690
485,ARROCHAR-SHORE ACRES,02 TWO FAMILY HOMES,521250,440039,81211
486,BLOOMFIELD,01 ONE FAMILY HOMES,500000,865000,-365000
